In [1]:
import sys
import os
import time
import numpy as np

In [2]:
import sys
import os
import time
import numpy as np

In [3]:
import pydrake
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra, IrisInConfigurationSpace, IrisOptions
from pydrake.common import FindResourceOrThrow
from pydrake.geometry import SceneGraph
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.optimization import CalcGridPointsOptions, Toppra
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.tree import RevoluteJoint
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.solvers.mosek import MosekSolver
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.primitives import TrajectorySource
from pydrake.trajectories import PiecewisePolynomial
from pydrake.all import Variable
from pydrake.all import MultibodyPositionToGeometryPose, ConnectMeshcatVisualizer, Role, Sphere
from pydrake.all import (
    ConvexSet, HPolyhedron, Hyperellipsoid,
    MathematicalProgram, Solve, le, IpoptSolver,
    Role, Sphere,
    Iris, IrisOptions, MakeIrisObstacles, Variable
)
from pydrake.all import (
    eq, SnoptSolver,
    Sphere, Ellipsoid, GeometrySet,
    RigidBody_, AutoDiffXd, initializeAutoDiff,
)

import pydrake.symbolic as sym

from meshcat import Visualizer

# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

def xreplace(expr, rules):
        if isinstance(expr, float) or isinstance(expr, sym.Variable):
            return expr
        assert isinstance(expr, sym.Expression), expr
        for old, new in rules:
            if expr.EqualTo(old):
                return new
        ctor, old_args = expr.Deconstruct()
        new_args = [xreplace(e, rules) for e in old_args]
        return ctor(*new_args)
    
from pydrake.symbolic import RationalFunction

def generate_rationalize_trig_expr_rules(q_var, t_var):
    rules = []
    denom = 1
    for i in range(t.shape[0]):
        sin_rule = (sym.sin(q[i]), (2*t[i])/(1+t[i]**2))
        cos_rule = (sym.cos(q[i]), (1-t[i]**2)/(1+t[i]**2))
        rules += [sin_rule, cos_rule]
#         denom *= (1+t[i]**2)
    return rules


def rationalize_trig_expr(expr, rules):
    return xreplace(expr, rules)

In [4]:
q = sym.MakeVectorContinuousVariable(3,"q")
prog = MathematicalProgram()
t = prog.NewContinuousVariables(q.shape[0], "t")

x1 = sym.sin(q[0])
y1 = -sym.cos(q[1])
x2 = sym.sin(q[0])+ 1/2*sym.sin(q[1])
y2 = sym.cos(q[0]) + sym.cos(q[1])

In [5]:
sub_rules = generate_rationalize_trig_expr_rules(q,t)
u1 = rationalize_trig_expr(x1, sub_rules)
v1 = rationalize_trig_expr(y1, sub_rules)
u2 = rationalize_trig_expr(x2, sub_rules)
v2 = rationalize_trig_expr(y2, sub_rules)

In [6]:
print(u2)

(((2 * t(0)) / (1 + pow(t(0), 2))) + 0.5 * ((2 * t(1)) / (1 + pow(t(1), 2))))


In [26]:
class NotRationalFunctionException(Exception):
    pass

import numbers
from multiprocess import Pool

pool = Pool(4)
def make_rational_function_from_expression(expr):
    # TODO handle nested fractions
    LegalPolyExpressionKind = [
        sym.ExpressionKind.Var,
        sym.ExpressionKind.Add,
        sym.ExpressionKind.Mul,
        sym.ExpressionKind.Div,
        sym.ExpressionKind.Pow,
        sym.ExpressionKind.Constant
    ]
    if isinstance(expr, (numbers.Number, np.number)):
        return sym.Polynomial(expr)


    expr_kind = expr.get_kind()
    
    if expr.is_polynomial():
        return sym.Polynomial(expr)
    elif expr_kind not in LegalPolyExpressionKind:
        raise NotRationalFunctionException(expr.to_string() + " is not rational")
    elif expr_kind == sym.ExpressionKind.Div:
        (ctor, (numerator, denominator)) = expr.Deconstruct()
        numerator = sym.Polynomial(numerator)
        denominator = sym.Polynomial(denominator)
        return RationalFunction(numerator, denominator)
    elif expr_kind in LegalPolyExpressionKind:
        (ctor, args) = expr.Deconstruct()
        if expr_kind == sym.ExpressionKind.Mul or expr_kind == sym.ExpressionKind.Pow:
            res = RationalFunction(1)
        else:
            res = RationalFunction(0)
        print(recursive_rets[0])
        #[Parallel(n_jobs = -1)(
#             delayed(make_rational_function_from_expression)(e) for e in args)]
        recursive_rets = [r.get() for r in recursive_rets]
        print(recursive_rets)
        return ctor(*recursive_rets)                                    
    
    else:
        raise NotRationalFunctionException(expr.to_string() + " is not rational but of type " + expr.get_kind())

    
# print(u2)
u2_rat = make_rational_function_from_expression(u2)
# print(u2_rat)
print(u2_rat.numerator())
print()
print(u2_rat.denominator())

TypeError: cannot pickle 'pydrake.symbolic.Expression' object

In [10]:
tmp = RationalFunction(u2_rat.numerator(), u2_rat.denominator())
print(u2_rat == tmp)

True


In [11]:
print("done")

done
